In [1]:
import pandas as pd

In [44]:
desc_df = pd.read_excel("../data/desc.xlsx")

In [45]:
desc_df.head()

,project_id,project_address,desc_total,video_desc,risk_desc
0,804380059,/projects/1552697291/ufactory-3d-printing-shop/,Follow us on Facebook and Twitter 횂혻and pl...,my name is Matt Elkins not here to tell you a...,Risks and challenges The biggest challenge of...
1,1560420632,/projects/941562770/nervous-dating-dating-for-...,"""I get very nervous when I think someone is f...",introverts the idea of going on the site can ...,Risks and challenges Fulfilment of your 'free...
2,278626358,/projects/703843051/messenger-rock/,"Hi, do you feel an inner hatred that you wish...",NaN,Risks and challenges The rock quality. Learn ...
3,907761690,/projects/1407693875/wind-turbine-phone-charge...,The goal of my project is to build a wind tur...,NaN,Risks and challenges The risks and challenges...
4,1512099429,/projects/358359447/crash-surveillance-drone-csd/,The project will consist of building a protot...,NaN,Risks and challenges The initial project is t...


In [46]:
new_desc_df = desc_df[['project_id', 'project_address']]

In [47]:
from bs4 import BeautifulSoup
from bleach import clean

def strip_html(fpath, allowed=['p']):
    try:
        text = open(fpath, "r").read()

        if 'has been hidden for privacy' in text:
            return ""
        elif 'this project is no longer available' in text:
            return ""
        elif 'is the subject of an intellectual property dispute' in text:
            return ""

        text = text.partition("Story</h3>")[2]
        text = text.partition("risks-and-challenges")[0]

        soup = BeautifulSoup(text, 'html.parser')    

        cleaned_text = clean(soup, tags=allowed, strip=True, strip_comments=True)
        #cleaned_text.encode('ascii',errors='ignore')
        cleaned_text = cleaned_text.replace(u'\xa0',u' ')
        cleaned_text = cleaned_text.replace(u'\n',u' ')
        cleaned_text = cleaned_text.replace(u'<p>',u' ')    
        cleaned_text = cleaned_text.replace(u'</p>',u' ')

        if cleaned_text[0] == ' ':
            cleaned_text = cleaned_text[1:]

        return cleaned_text
    except:
        print(fpath)
        return ""        

In [48]:
import tqdm
import os
import codecs
import time

cwd = os.getcwd()

desc_list = []

for i, r in tqdm.tqdm(new_desc_df.iterrows(), total=new_desc_df.shape[0]):
    desc_dict = {}
    desc_dict['project_address'] = r['project_address']
    
    fpath = cwd + "/html/{}.html".format(i)
    text = strip_html(fpath, allowed=['p'])
    
    desc_dict['desc_total_renewal'] = text
    
    desc_list.append(desc_dict)

 61%|██████    | 4592/7571 [13:31<09:22,  5.30it/s]

/Users/jonghkim/dev_jhk/Research/Crowdfunding/Speech_Emotion/risk-perception/notebook/html/4592.html


 78%|███████▊  | 5871/7571 [16:42<02:42, 10.48it/s]

/Users/jonghkim/dev_jhk/Research/Crowdfunding/Speech_Emotion/risk-perception/notebook/html/5868.html


 79%|███████▉  | 6010/7571 [16:58<05:43,  4.55it/s]

/Users/jonghkim/dev_jhk/Research/Crowdfunding/Speech_Emotion/risk-perception/notebook/html/6010.html


100%|██████████| 7571/7571 [20:48<00:00,  9.24it/s]


In [58]:
desc_renewal_df = pd.DataFrame(desc_list)

In [59]:
desc_renewal_df.head()

,desc_total_renewal,project_address
0,Follow us on Facebook and Twitter and please s...,/projects/1552697291/ufactory-3d-printing-shop/
1,"""I get very nervous when I think someone is fa...",/projects/941562770/nervous-dating-dating-for-...
2,"Hi, do you feel an inner hatred that you wish ...",/projects/703843051/messenger-rock/
3,The goal of my project is to build a wind turb...,/projects/1407693875/wind-turbine-phone-charge...
4,The project will consist of building a prototy...,/projects/358359447/crash-surveillance-drone-csd/


In [70]:
risk_desc_df = pd.read_excel('risk_desc_df.xlsx')

In [71]:
risk_desc_df.head()

,project_id,project_address,text
0,804380059,/projects/1552697291/ufactory-3d-printing-shop/,The biggest challenge of my project will be to...
1,1560420632,/projects/941562770/nervous-dating-dating-for-...,Fulfilment of your 'free membership codes' won...
2,278626358,/projects/703843051/messenger-rock/,The rock quality.
3,907761690,/projects/1407693875/wind-turbine-phone-charge...,The risks and challenges that may arise during...
4,1512099429,/projects/358359447/crash-surveillance-drone-csd/,The initial project is to demonstrate a proof ...


In [72]:
desc_total_df = desc_renewal_df.merge(risk_desc_df, on='project_address', how='left')

In [73]:
desc_total_df.head()

,desc_total_renewal,project_address,project_id,text
0,Follow us on Facebook and Twitter and please s...,/projects/1552697291/ufactory-3d-printing-shop/,804380059,The biggest challenge of my project will be to...
1,"""I get very nervous when I think someone is fa...",/projects/941562770/nervous-dating-dating-for-...,1560420632,Fulfilment of your 'free membership codes' won...
2,"Hi, do you feel an inner hatred that you wish ...",/projects/703843051/messenger-rock/,278626358,The rock quality.
3,The goal of my project is to build a wind turb...,/projects/1407693875/wind-turbine-phone-charge...,907761690,The risks and challenges that may arise during...
4,The project will consist of building a prototy...,/projects/358359447/crash-surveillance-drone-csd/,1512099429,The initial project is to demonstrate a proof ...


In [74]:
desc_total_df.columns = ['desc_total', 'project_address', 'project_id', 'risk_desc']

In [77]:
desc_total_df = desc_total_df[['project_id', 'project_address', 'desc_total','risk_desc']]

In [78]:
desc_total_df.to_pickle('desc_total_df_renewal.pkl')

In [79]:
desc_total_df.to_csv('desc_total_df_renewal.csv', encoding='utf-8')